In [ ]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                  image                   id  \
1383  ../../data/Train/CameraRGB/episode_0009_000166...  episode_0009_000166   
3354  ../../data/Train/CameraRGB/episode_0001_000216...  episode_0001_000216   
2753                 ../../data/Train/CameraRGB/832.png                  832   
491   ../../data/Train/CameraRGB/episode_0007_000217...  episode_0007_000217   
401                  ../../data/Train/CameraRGB/144.png                  144   

                                                  label  
1383  ../../data/Train/CameraSeg/episode_0009_000166...  
3354  ../../data/Train/CameraSeg/episode_0001_000216...  
2753                 ../../data/Train/CameraSeg/832.png  
491   ../../data/Train/CameraSeg/episode_0007_000217...  
401                  ../../data/Train/CameraSeg/144.png  


In [4]:
from models.fcn8 import model_fcn8

model = model_fcn8(3, image_shape=(384, 384, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 384, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 384, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 192, 192, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [5]:
from gen.datagen import balanced_generator_from_df

BATCH_SIZE = 32

model_dir = '../../saved_models/fcn8/fcn8_v6/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)


train_gen = balanced_generator_from_df(train_df, BATCH_SIZE, (384, 384))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (384, 384))


In [6]:
from train import iou_loss
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import RMSprop

gpus = 2

# opt = RMSprop(lr=1e-04)
model_gpu = multi_gpu_model(model, gpus)
model_gpu.compile(loss=iou_loss,
              optimizer='adam',
              metrics=['acc'])


In [7]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/100
75/75 [==============================] - 74s 985ms/step - loss: 0.2091 - acc: 0.7924 - val_loss: 0.0570 - val_acc: 0.9438

Epoch 00001: val_loss improved from inf to 0.05699, saving model to ../../saved_models/fcn8/fcn8_v6//model.hdf5
Epoch 2/100
75/75 [==============================] - 71s 945ms/step - loss: 0.0392 - acc: 0.9617 - val_loss: 0.0328 - val_acc: 0.9681

Epoch 00002: val_loss improved from 0.05699 to 0.03275, saving model to ../../saved_models/fcn8/fcn8_v6//model.hdf5
Epoch 3/100
75/75 [==============================] - 70s 938ms/step - loss: 0.0292 - acc: 0.9717 - val_loss: 0.0266 - val_acc: 0.9743

Epoch 00003: val_loss improved from 0.03275 to 0.02658, saving model to ../../saved_models/fcn8/fcn8_v6//model.hdf5
Epoch 4/100
75/75 [==============================] - 70s 930ms/step - loss: 0.0259 - acc: 0.9750 - val_loss: 0.0266 - val_acc: 0.9743

Epoch 00004: val_loss did not improve
Epoch

75/75 [==============================] - 70s 932ms/step - loss: 0.0059 - acc: 0.9952 - val_loss: 0.0100 - val_acc: 0.9910

Epoch 00038: val_loss did not improve
Epoch 39/100
75/75 [==============================] - 70s 940ms/step - loss: 0.0060 - acc: 0.9951 - val_loss: 0.0104 - val_acc: 0.9906

Epoch 00039: val_loss did not improve
Epoch 40/100
18/75 [======>.......................] - ETA: 47s - loss: 0.0059 - acc: 0.9953

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show(

In [8]:
# model_gpu.load_weights('../../saved_models/fcn8/fcn8_v6/model.hdf5')
model.save('../../saved_models/fcn8/fcn8_v6/fcn8_v6.h5')

In [ ]:
#valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE * gpus)
#test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE * gpus)